In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [2]:
%load_ext dotenv
%dotenv

In [3]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-06-01",
    temperature=0.7
)


In [11]:
system_prompt_optimistic = SystemMessagePromptTemplate.from_template("ユーザの入力に対して楽観的な意見をください")
system_prompt_pessimistic = SystemMessagePromptTemplate.from_template("ユーザの入力に対して悲観的な意見をください")
user_prompt = HumanMessagePromptTemplate.from_template("{query}")
template_optimistic = ChatPromptTemplate.from_messages([
    system_prompt_optimistic,
    user_prompt
])
template_pessimistic = ChatPromptTemplate.from_messages([
    system_prompt_pessimistic,
    user_prompt
])

In [12]:
output_parser = StrOutputParser()

In [17]:
import pprint
from langchain_core.runnables import RunnableParallel
from operator import itemgetter
chain_optimistic = template_optimistic | llm | output_parser
chain_pessimistic = template_pessimistic | llm | output_parser
parallel = RunnableParallel(
    {
        "chain_optimistic": chain_optimistic,
        "chain_pessimistic": chain_pessimistic,
        "aspect": itemgetter("aspect")
    }
)

# output = parallel.invoke({"query": "宝くじで１００万円当選した"})
# print(output)

In [ ]:
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "２つの意見を{aspect}の観点からまとめてください"),
    ("user", "楽観的: {chain_optimistic}\n悲観的: {chain_pessimistic}")
])

In [18]:
chain = parallel | summary_prompt | llm | output_parser
output = chain.invoke({"query": "宝くじで１００万円当選した", "aspect": "経済的"})
print(output)

宝くじで100万円当選したというニュースには、楽観的と悲観的な視点があります。楽観的な意見では、100万円の当選は大きな喜びであり、日頃の努力や運の結果として人生を少し豊かにする素晴らしいチャンスと捉えられています。この金額を使って旅行や買い物、貯金などを楽しむことができ、自分へのご褒美として贅沢を味わうことも推奨されています。そして、この幸運がさらなる良い出来事の始まりになることを期待しています。

一方で、悲観的な意見では、100万円が実際にはそれほど大きな額ではない可能性を指摘しています。税金や生活費、将来の備えを考えると、その金額は限定的であり、計画的に使わないと簡単に消えてしまうリスクがあると警告しています。また、宝くじの当選をきっかけに、無駄遣いや宝くじへの依存が生じる可能性があり、長期的な経済的安定に寄与しないという懸念も示されています。

まとめると、100万円の当選は喜ばしいニュースであり、計画的に使えば人生を少し豊かにするチャンスとなりますが、注意を怠るとその価値を失い、経済的安定に悪影響を及ぼす可能性もあるため、慎重な判断が重要です。
